# Using the PySTAC API

There is an abundance of data searchable through NASA's [Earthdata Search](https://search.earthdata.nasa.gov). These are examples of [SpatioTemporal Asset Catalogs (STAC)](https://stacspec.org/) in that data is typically associated with a specific *Area of Interest (AOI)* and a *time-window* or *range of dates*.

For the sake of reproducibility, we want to search asset catalogs programmatically. This is where the [PySTAC](https://pystac.readthedocs.io/en/stable/) library comes in.

---

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

# data wrangling imports
import numpy as np
import pandas as pd
import xarray as xr
# STAC imports to retrieve cloud data
from pprint import pprint
from pystac_client import Client

In [ ]:
# Imports for plotting
import hvplot.pandas
import geoviews as gv
from geoviews import opts
gv.extension('bokeh')

## Defining AOI & range of dates

Heavy rains severely impacted Argentina in March 2024 [[1]](https://www.reuters.com/world/americas/argentina-downpour-drenches-crop-fields-flash-floods-buenos-aires-2024-03-12/). The event resulted in flash floods and impacted crop yields, severely impacting the Buenos Aires metropolitan area, and caused significant damage to property and human life. In this notebook, we'll set up a DataFrame to process results retrieved when searching relevant OPERA DSWx-HLS data catalogs.

In [ ]:
# Define data search parameters

# Define AOI as left, bottom, right and top lat/lon extent
aoi = (-59.63818, -35.02927, -58.15723, -33.77271)
# We will search data for the month of March 2024
date_range = '2024-03-01/2024-03-31'

Make a quick visual check that the tuple `aoi` actually describes the geographic region around Buenos Aires.

In [ ]:
basemap = gv.tile_sources.OSM
rect = gv.Rectangles(aoi).opts(opts.Rectangles(alpha=0.25, color='cyan'))

rect*basemap

## Executing a search with the PySTAC API

In [ ]:
# We open a client instance to search for data, and retrieve relevant data records
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'

# Setup PySTAC client
catalog = Client.open(f'{STAC_URL}/POCLOUD/')
collections = ["OPERA_L3_DSWX-HLS_V1"]

# We would like to search data using the search parameters defined above.
opts = {
    'bbox' : aoi, 
    'collections': collections,
    'datetime' : date_range,
}

search = catalog.search(**opts)

In [ ]:
print(f'{type(search)=}')

In [ ]:
results = list(search.items_as_dicts())
print(f"Number of tiles found intersecting given bounding box: {len(results)}")

The object `results` retrieved from the search is a list of Python dictionaries (as suggested by the method name `items_as_dicts`). Let's parse the the first entry of `results`.

In [ ]:
result = results[0]
print(f'{type(result)=}')
print(result.keys())

Each element of `results` is a dictionary that contains other other nested dictionaries. The Python utility `pprint.pprint` library helps us examine the structure of the search results.

In [ ]:
pprint(result, compact=True, width=10, sort_dicts=False)

The particular values we want to pick out from `result` are:
+ `result['properties']['datetime']` : timestamp associated with a particular granule; and
+ `result['assets']['0_B01_WTR']['href']` : URI associated with a particular granule (pointing to a GeoTIFF file).

```
{...
 'properties': {'eo:cloud_cover': 95,
                'datetime': '2024-03-01T13:44:11.879Z',
                'start_datetime': '2024-03-01T13:44:11.879Z',
                'end_datetime': '2024-03-01T13:44:11.879Z'},
 'assets': {'0_B01_WTR': {'href': 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/OPERA_L3_DSWX-HLS_PROVISIONAL_V1/OPERA_L3_DSWx-HLS_T21HUC_20240301T134411Z_20240305T232837Z_L8_30_v1.0_B01_WTR.tif',
                          'title': 'Download '
                                   'OPERA_L3_DSWx-HLS_T21HUC_20240301T134411Z_20240305T232837Z_L8_30_v1.0_B01_WTR.tif'},
            '0_B02_BWTR': ...
            }

```

In [ ]:
# Look at specific values extracted from the 'properties' & 'assets' keys.
print(result['properties']['datetime'])
print(result['assets']['0_B01_WTR']['href'])

## Summarizing search results in a DataFrame

Let's extract these particular fields into a Pandas DataFrame for convenience.

In [ ]:
times = pd.DatetimeIndex([result['properties']['datetime'] for result in results])
hrefs = {'hrefs': [result['assets']['0_B01_WTR']['href'] for result in results]}

In [ ]:
# Construct Pandas DataFrame to summarize granules from search results
granules = pd.DataFrame(index=times, data=hrefs)
granules.index.name = 'times'

In [ ]:
granules

In [ ]:
len(granules.index.unique()) / len(granules) # Notice the timestamps are not all unique, i.e., some are repeated

In [ ]:
len(granules.hrefs.unique()) / len(granules) # Make sure all the hrefs are unique

Let's get a sense of how many granules are available for each day of the month. Note, we don't know how many of these tiles contain cloud cover obscuring features of interest yet.

The next few lines do some manipulations in Pandas of the DataFrame `granules` to yield a line plot showing what dates are associated with the most granules.

In [ ]:
granules_by_day = granules.resample('1d')  # Grouping by day, i.e., "resampling"

In [ ]:
granule_counts = granules_by_day.count() # Aggregating counts

In [ ]:
# Ignore the days with no associated granules
granule_counts = granule_counts[granule_counts.hrefs > 0]

In [ ]:
# Relabel the index & column of the DataFrame
granule_counts.index.name = 'Day of Month'
granule_counts.rename({'hrefs':'Granule count'}, inplace=True, axis=1)

In [ ]:
count_title = '# of DSWx-HLS granules available / day'
granule_counts.hvplot.line(title=count_title, grid=True, frame_height=300, frame_width=600)

The floods primarily occurred between March 11th and 13th. Unfortunately, there are few granules associated with those particular days. We can, in principal, use the URIs stored in this DataFrame to set up analysis of the data associated with this event; we'll do so in other examples with better data available.

---

In subsequent notebooks, we'll use this general workflow:

1. Set up a search query by identifying a particular AOI and range of dates.
2. Identify a suitable asset catalog and execute the search using `pystac.Client`.
3. Convert the search results into a Pandas DataFrame containing the principal fields of interest.
4. Use the resulting DataFrame to access relevant remote data for analysis and/or visualization.

In [ ]:
granules_by_day # Recall our earlier resampling by day

In [ ]:
# Aggregate all granules by day into a list for each day
granules_by_day = granules_by_day['hrefs'].apply(list)

In [ ]:
granules_by_day.head(5).map(len) # Should match granule_counts from earlier

In [ ]:
# Note: these dates do not agree with the text in the Markdown block above; fix?
dates_of_interest = ['2024-03-01', '2024-03-17', '2024-03-28']

In [ ]:
# The Pandas Series granules_by_day returns a list of URIs for each date; these can be merged into images. 
granules_by_day.loc[dates_of_interest[-1]]

In [ ]:
# Assemble DataArray from merged data from those three days.
# This could take some time to download TIF files (within the function rasterio.merge.merge)

# Note: the computation below of x_coords & y_coords may be incorrect; the formulas are inferred from https://rasterio.readthedocs.io/en/latest/quickstart.html.
# It is also possible that they can be computed once outside the loop rather than repeatedly.
da_list, transform_list = [], []
for date in dates_of_interest:
    merged_array, transform = merge(granules_by_day.loc[date])
    dims = merged_array.squeeze().shape
    x_min, y_max = transform * (0,0)
    x_max, y_min = transform * dims
    x_coords = np.linspace(x_min, x_max, dims[0])
    y_coords = np.linspace(y_min, y_max, dims[1])
    da_list.append( xr.DataArray(data=merged_array,
                                       coords=[('t',[date]), ('x', x_coords), ('y', y_coords)]) )
    transform_list.append(transform)

In [ ]:
da = xr.concat(da_list, dim='t')

One interesting observation: the images are stored as 8-bit unsigned integers (i.e., values between 0 and 255).

Although there are 256 possible values, there are only a handful of distinct values in the dynamic range of the numerical data (as we can see by extracting the pixel values into a Pandas Series).

In [ ]:
pixel_values = pd.Series(da.values.flatten())
pixel_values.value_counts().sort_index() / len(pixel_values) # fewer than a half dozen distinct pixel values.

The pixel values 1, 2, & 252 correspond to water; the values 0 & 255 correspond either to land or missing data.

To help visualize the data, we first extract the colormap from one of the granules. We then flag the constraints above by assigning the corresponding pixel values blue or white (to be treated as transparent).

In [ ]:
# Pick the first file for the first key available
first_file = granules_by_day.loc[dates_of_interest[0]][0]
# Load up color maps for visualization
with rasterio.open(first_file) as src:
    colormap = src.colormap(1)
    crs = src.crs

In [ ]:
# Convert colormap from dict to NumPy array while rescaling values to [0,1]
colormap = np.concatenate([ np.array([colormap[k]]) / 255 for k in colormap ], axis=0)

In [ ]:
# Assign the
blue_bands, transparent_bands = [1, 2, 252], [0, 255]
colormap[blue_bands,:] = [0,0,1,1]
colormap[transparent_bands,:] = [0,0,0,0]

In [ ]:
colormap = ListedColormap(colormap)

In Matplotlib, these three time slices can be plotted with the code below.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (30, 10))

for i in range(3):
    show(da.isel(t=i).values, ax=ax[i], cmap=colormap, transform=transform_list[i], interpolation=None)

The above plots can be improved with some additional code; it's a little tricky to read.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (30, 10))

for i in range(3):
    show(da.isel(t=i).values, ax=ax[i], cmap=colormap, transform=transform_list[i], interpolation=None)
    #cx.add_basemap(ax[i], crs=crs, zoom=10, source=cx.providers.OpenStreetMap.Mapnik)
    show(da.isel(t=i).values, ax=ax[i], cmap=colormap, transform=transform_list[i], interpolation=None)

    scalebar = AnchoredSizeBar(ax[i].transData,
                            5000, '5 km', 'lower right', 
                            color='black',
                            frameon=False,
                            pad = 0.25,
                            sep=5,
                            fontproperties = {'weight':'semibold', 'size':12},
                            size_vertical=300)

    ax[i].add_artist(scalebar)
    ax[i].ticklabel_format(axis='both', style='scientific',scilimits=(0,0),useOffset=False,useMathText=True)
    ax[i].set_xlabel('UTM easting (meters)')
    ax[i].set_ylabel('UTM northing (meters)')
    ax[i].set_title(f"Water extent on: {da.coords['t'].values[i]}")

Alternatively, the Hvplot library provides ways to plot data directly from Pandas DataFrames or Xarray DataArrays.

In [ ]:
cmap = process_cmap(colormap) # Convert Matplotlib colormap into form suitable for Holoviz/Geoviews/Hvplot

In [ ]:
da.hvplot.image(x='x', y='y', colorbar=True, cmap=cmap, aspect='equal',  datashade=True,
                 frame_width=150, frame_height=150, dynamic=True,).opts(xlabel='UTM easting', ylabel='UTM northing', alpha=0.5,)

In [ ]:
da.hvplot.quadmesh(x='x', y='y', colorbar=True, cmap=cmap, aspect='equal',  datashade=True,
                 frame_width=150, frame_height=150, dynamic=True,).opts(xlabel='UTM easting', ylabel='UTM northing', alpha=0.5,)

In [ ]:
da.hvplot.quadmesh(x='x', y='y', cmap=cmap, aspect='equal', colorbar=False, datashade=True,
                 frame_width=100, frame_height=100, dynamic=True,).opts(xlabel='UTM easting', ylabel='UTM northing', alpha=1,)